# Описание проекта
Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.
## Описание данных
*Признаки:* пол, возраст и зарплата застрахованного, количество членов его семьи.
*Целевой признак:* количество страховых выплат клиенту за последние 5 лет.
## Постановка задачи
### [Шаг 1. Знакомство с данными, их изучение и предобработка](#section1)
Загрузите и изучите данные.
### [Шаг 2. Работа с матрицами:](#section2)
Ответьте на вопрос и обоснуйте решение.
 Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
1. Изменится. Приведите примеры матриц.
2. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
### [Шаг 3. Построение алгоритма преобразований:](#section3)
Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
### [Шаг 4. Оценка качества алгоритма:](#section4)
Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.
### [Шаг 5. Общий вывод](#section5)


# Решение задачи
## Шаг 1. Знакомство с данными, их изучение и предобработка <a class="anchor" id="section1"></a>

Импортируем необходимые библиотеки для дальнейшей работы.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import warnings

warnings.filterwarnings('ignore')

RANDOM_VAL = np.random.RandomState(42)

In [2]:
insurance_data = pd.read_csv('insurance.csv')

In [3]:
# Функция изучения данных в таблице
def research_info(data):
    rows_count, columns_count = data.shape
    print('Количество стобцов:', columns_count)
    print('Количество строк:', rows_count)
    print('Дубликатов: {} ({:.2%})'.format(data.duplicated().sum(), data.duplicated().sum() / len(data)))
    print('Пропусков: {}'.format(data.isna().sum().sum()))
    data.info()
    display(data.sample(n=5))

In [4]:
research_info(insurance_data)

Количество стобцов: 5
Количество строк: 5000
Дубликатов: 153 (3.06%)
Пропусков: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
3406,0,41.0,42100.0,1,0
257,0,37.0,52500.0,3,0
3032,1,37.0,22900.0,3,0
1623,1,38.0,41700.0,1,0
4682,0,34.0,45100.0,0,0


In [5]:
# insurance_data.drop_duplicates(inplace=True)

**Вывод:**
- пропусков нет;
- дубликаты удалены.

Данные готовы к дальнейшей работе.

## Шаг 2. Работа с матрицами <a class="anchor" id="section2"></a>

Обозначения:

— $X$ матрица признаков (нулевой столбец состоит из единиц);
— $y$ вектор целевого признака;
— $P$ матрица, на которую умножаются признаки;
— $w$ вектор весов линейной регрессии (нулевой элемент равен сдвигу).

Вектор предсказаний $a$ линейной регрессии можно описать формулой: $\[a = X*w\]$
Тогда задача обучения имеет вид: $\[w = arg \min_w MSE(Xw, y)\]$
Формула обучения: $\[w=(X^{T}X)^{-1}X^{T}y\]$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится.

**Обоснование:**

Умножим матрицу признаков $X$ на матрицу $P$ и получим новую матрицу $X_P = XP$, которую подставим в формулу обучения:
$\[w_P=((XP)^{T}(XP))^{-1}(XP)^{T}y\]$
Упростим её:
$\[w_P=((XP)^{T}(XP))^{-1}(XP)^{T}y =\]$
$\[ = [P^TX^TXP]^{-1}P^TX^Ty =\]$
$\[ = P^{-1}[P^T(X^TX)]^{-1}P^TX^Ty =\]$
$\[ = P^{-1}(X^TX)^{-1}[(P^T)^{-1}P^T]X^Ty =\]$
$\[ = P^{-1}(X^TX)^{-1}X^Ty\]$,
где $\[(P^T)^{-1}P^T = E\]$, т.е. единичной матрице, а значит связь параметров линейной регрессии в исходной задаче и в преобразованной можно выразить как: $\[w_P = P^{-1}w(X)\]$
Полученные коэффициенты подставим в формулу предсказания и упростим выражение:
$\[a_P = X_P*w_P = XP[P^{-1}w(X)] = Xw\]  $

После выполнения указанных преобразований получено взаимно-однозначное соответствие параметров исходной и преобразованной задачи. Значит, значение предсказания $a$ не меняется, если умножать матрицу признаков на обратимую матрицу.

Проверим, что линейная регрессия из scikit-learn вычисляет значения по такому же правилу.

In [6]:
features = insurance_data.drop('Страховые выплаты', axis=1)
target = insurance_data['Страховые выплаты']

In [7]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])
model = LinearRegression()
model.fit(features, target)
display(model.coef_)

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

Коэфициенты регрессии совпадают на 100%.

**Вывод:**
Качество линейной регрессии не меняется при умножении матрицы признаков на обратимую матрицу.

In [8]:
# Сгенерируем случайную квадратную матрицу P с размером, равным количеству признаков,
# (т.к. количество столбцов исходной матрицы датасета должно совпадать с количеством строк матрицы P):
P = np.random.normal(size=(features.shape[1], features.shape[1]))
display(P)

array([[ 2.04349631,  1.23267328,  0.33516589, -0.52307541],
       [ 0.02431496, -0.7312016 , -0.15113296, -1.12510237],
       [ 0.69265194, -0.73495964, -1.03044089,  0.69816644],
       [ 1.00316727, -0.91586763,  0.32573849,  0.85949784]])

Чтобы найти обратную матрицу, вызовем функцию numpy.linalg.inv() (от англ. linear algebra, «линейная алгебра»; invert, «обратить»). Также она поможет проверить матрицу на обратимость: если матрица необратима, будет обнаружена ошибка.

In [9]:
# Проверим, что эта матрица обратима (умножим её на обратную ей матрицу):
display(np.round(P @ np.linalg.inv(P)))

array([[ 1.,  0.,  0.,  0.],
       [-0.,  1.,  0.,  0.],
       [ 0.,  0.,  1., -0.],
       [-0.,  0.,  0.,  1.]])

По условию задания необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и, при этом, качество моделей машинного обучения не ухудшилось.

Преобразуем данные с помощью умножения на случайную матрицу и сравним с исходной.

In [10]:
# Умножим признаки обучающей выборки исходного датасета на сгенерированную случайную квадратную матрицу P:
features_P = features @ P
target_P = target
display(features_P.head())

,0,1,2,3
0,34359.579932,-36483.660597,-51115.403859,34583.262846
1,26322.895476,-27963.017456,-39163.380326,26479.429660
2,14546.395929,-15455.357283,-21643.641617,14628.867355
3,28886.102957,-30665.003951,-42971.907569,29091.632560
4,18080.940018,-19201.687572,-26898.403874,18190.118246


После преобразования данные перестали нести в себе четкий смысл и потеряли однозначность трактовки. Для их дешифровки необходимо иметь доступ к матрице, с помощью которой и были совершены преобразования. Проверим, возможно ли восстановить данные при наличии исходной матрицы:

In [11]:
# Преобразованная матрица:
display(features.head())
display(np.round(features_P @ np.linalg.inv(P)).head())

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,0.0,46.0,38000.0,1.0
2,-0.0,29.0,21000.0,-0.0
3,0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,-0.0


## Шаг 4. Оценка качества алгоритма <a class="anchor" id="section4"></a>


In [12]:
r2_score_list = []
# Модель без преобразований
model = LinearRegression()
model.fit(features, target)
r2_score_list.append(['Линейная регрессия', r2_score(target, model.predict(features))])

# Модель без преобразования с отмасштабированными признаками
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([('standard_scaller', scaller), ('linear_regression', regressor)])
pipeline.fit(features, target)
r2_score_list.append(['Линейная регрессия c масштабом', r2_score(target, pipeline.predict(features))])

# Модель с умножением на матрицу
model_P = LinearRegression()
model_P.fit(features_P, target_P)
r2_score_list.append(['Линейная регрессия с умножением на матрицу', r2_score(target_P, model_P.predict(features_P))])

# Модель без преобразования с отмасштабированными признаками
regressor_P = LinearRegression()
scaller_P = StandardScaler()
pipeline_P = Pipeline([('standard_scaller', scaller_P), ('linear_regression', regressor_P)])
pipeline_P.fit(features_P, target_P)
r2_score_list.append(
    ['Линейная регрессия с умножением на матрицу и масштабом', r2_score(target_P, pipeline_P.predict(features_P))])

In [13]:
result_score = pd.DataFrame(r2_score_list, columns=['Тип модели', 'Оценка R2'])
result_score

,Тип модели,Оценка R2
0,Линейная регрессия,0.424946
1,Линейная регрессия c масштабом,0.424946
2,Линейная регрессия с умножением на матрицу,0.424946
3,Линейная регрессия с умножением на матрицу и м...,0.424946


**Вывод:**
В результате выполнения задач этого раздела было выявлено следующее:
- применив метрику R2, мы проверили, что качество линейной регрессии из sklearn не отличается до и после преобразования исходной матрицы;
- при использовании матриц для шифровки данных, есть возможность восстановить данные к исходному состоянию;
- при использовании совместно умножения на произвольную матрицу с последующим применением масштабирования понять что именно содержат данные не представляется возможным.

## Шаг 5. Общий вывод <a class="anchor" id="section5"></a>

Разработан такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Входные данные - данные клиентов страховой компании «Хоть потоп».

Результаты исследования позволят защитить данные клиентов страховой компании «Хоть потоп».

Перед проведением исследования была сформулирована гипотеза: при умножении признаков на обратимую матрицу качество линейной регрессии не меняется.
В результате удалось получить следующие результаты.

Гипотеза подтверждена:
0.43 - значение метрики R2 модели для исходных данных;
0.43 - значение метрики R2 модели для исходных данных с применением масштабирования;
0.43 - значение метрики R2 модели для преобразованных данных;
0.43 - значение метрики R2 модели для преобразованных с применением масштабирования.